In [1]:
# 1) Repo'yu klonla
!git clone https://github.com/guzelTemli/YapayZekaChatbot.git

Cloning into 'YapayZekaChatbot'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 290 (delta 9), reused 18 (delta 5), pack-reused 268 (from 2)
Receiving objects: 100% (290/290), 573.88 MiB | 17.63 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [2]:
#Huggingface'e bağlanma
from huggingface_hub import login
login()

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

BASE = "google/gemma-2-2b-it"

base_tokenizer = AutoTokenizer.from_pretrained(BASE)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

def ask_base(q):
    messages = [{"role": "user", "content": q}]
    prompt = base_tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = base_tokenizer(prompt, return_tensors="pt").to(base_model.device)

    with torch.no_grad():
        outputs = base_model.generate(
            **inputs,
            max_new_tokens=250,
            do_sample=True,
            temperature=0.4,
            top_p=0.9
        )

    answer_ids = outputs[0][inputs["input_ids"].shape[-1]:]
    answer = base_tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer.strip()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

BASE = "google/gemma-2-2b-it"
ADAPTER = "/content/YapayZekaChatbot/FINE-TUNE/gemma-2b-train-qlora-final"

ft_tokenizer = AutoTokenizer.from_pretrained(ADAPTER, use_fast=True)

base = AutoModelForCausalLM.from_pretrained(
    BASE,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

ft_model = PeftModel.from_pretrained(base, ADAPTER)

ft_tokenizer.pad_token = ft_tokenizer.eos_token
ft_model.config.pad_token_id = ft_tokenizer.eos_token_id
ft_model.eval()

def ask_ft(q):
    messages = [{"role": "user", "content": q}]
    prompt = ft_tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = ft_tokenizer(prompt, return_tensors="pt").to(ft_model.device)

    with torch.no_grad():
        outputs = ft_model.generate(
            **inputs,
            max_new_tokens=250,
            do_sample=True,
            temperature=0.4,
            top_p=0.9
        )

    answer_ids = outputs[0][inputs["input_ids"].shape[-1]:]
    answer = ft_tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer.strip()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
questions = [
    "Güçlü parola politikası nasıl olmalıdır?",
    "Uzaktan çalışma sırasında VPN neden gereklidir?",
    "Ayrıcalıklı hesap yönetimi neden kritik bir kontroldür?",
    "USB / çıkarılabilir medya kullanımı nasıl sınırlandırılmalıdır?",
    "Log yönetimi neden bilgi güvenliği için zorunludur?",
    "Kurum içi erişim yetkileri nasıl atanmalı ve izlenmelidir?"
]

for q in questions:
    print("\n" + "="*80)
    print("Soru:", q)
    print("\n--- BASE MODEL CEVABI ---")
    print(ask_base(q))
    print("\n--- FINE-TUNE MODEL CEVABI ---")
    print(ask_ft(q))



Soru: Güçlü parola politikası nasıl olmalıdır?

--- BASE MODEL CEVABI ---
Güçlü bir parola politikası, güvenlik ve gizlilik açısından önemli bir rol oynar. İşte güçlü bir parola politikası oluşturmak için bazı önemli noktalar:

**1. Parola Güçlü Olmalı:**

* **Uzunluk:** Parolalar en az 12 karakterden daha uzun olmalı. 
* **Karakter Tipleri:**  A, Z, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, #, $, %, & gibi karakterleri içermelidir.
* **Kelimeler:**  Parola kelimelerden oluşmalıdır. Kelimeleri birleştirerek daha güçlü bir parola oluşturabilirsiniz.
* **Döngü:** Parola bir dizi karakterden oluşmalıdır. 
* **Özel Karakterler:** Özel karakterler (şifreleme, semboller, harfler) ile daha güçlü bir parola oluşturmak için kullanılabilir.

**2. Parola Kullanımı:**

* **Her Kullanımda Yeni Parola:** Her yeni hesap için yeni bir parola oluşturmalısınız.
* **Parola Kaydetme:** Parolalarınızı güvenli

--- FINE-TUNE MODEL CEVABI ---
Parola uzunluk, karakter çeşitliliği, tekrarlama kısıtlamaları ve güncelleme 